Library****

In [ ]:
import pandas as pd
import numpy as np

> DATA

In [ ]:
train=pd.read_csv("../input/ncaaw-2021-features-only/tourney.csv")
test=pd.read_csv("../input/ncaaw-2021-features-only/test.csv")
train["result"].head()


In [ ]:
Y=train["result"]
S=train["Season"]
X=train.drop(['Season','TeamID1','TeamID2','result'],axis=1)
X_test=test.drop(['ID','TeamID1','TeamID2','Season'],axis=1)
Y.tail(3)

*Use H2O Randomforest estimator*

In [ ]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator

In [ ]:
h2o.init()

In [ ]:
h2o_train=h2o.H2OFrame(train)
h2o_test=h2o.H2OFrame(test)

In [ ]:
num_folds = 10
training_columns = list(X.columns)


Model****

In [ ]:
model = H2OGradientBoostingEstimator(ntrees=1000,
                                 max_depth=50,
                                 nfolds=num_folds,
                                 seed=1234)

> Train Model

In [ ]:
model.train(x=training_columns,
            y='result',
            training_frame=h2o_train)

predictions on test sample

In [ ]:
test_preds = model.predict(test_data=h2o_test).as_data_frame()['predict'].values

In [ ]:
MSampleSubmission = pd.read_csv('../input/ncaaw-march-mania-2021/WSampleSubmissionStage1.csv')

In [ ]:
idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test.ID, pd.Series(test_preds)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = MSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub.to_csv('submission.csv', index=False)
sub.head()